A simple example to try to demonstrate the benefit of GP over sample mean estimators.

Try to make a story, using this to help make up some numbers.

In [2]:
import torch
from botorch import fit_gpytorch_model
from botorch.models import FixedNoiseGP
from gpytorch import ExactMarginalLogLikelihood


noise_level = 0.5
true_mean = torch.tensor([[-1.], [0.], [1.]])

In [3]:
observations = true_mean + torch.randn(2, 3, 1) * noise_level
print(observations)

tensor([[[-0.3193],
         [-0.5994],
         [ 1.9475]],

        [[-2.1729],
         [-0.0986],
         [ 0.4242]]])


In [4]:
X = torch.tensor([[0.0], [0.5], [1.0]])

model = FixedNoiseGP(X.repeat(2, 1), observations.view(-1, 1), torch.ones(6, 1) * noise_level)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
_ = fit_gpytorch_model(mll)

In [5]:
model.posterior(X).mean, model.posterior(X).mvn.covariance_matrix

(tensor([[-1.1266],
         [-0.3106],
         [ 1.0280]], grad_fn=<UnsqueezeBackward0>),
 tensor([[ 0.2197,  0.0098, -0.0018],
         [ 0.0098,  0.2167,  0.0098],
         [-0.0018,  0.0098,  0.2197]], grad_fn=<AddBackward0>))

In [6]:
model.forward(X).mean, model.forward(X).covariance_matrix

(tensor([-0.1053, -0.1053, -0.1053], grad_fn=<ExpandBackward>),
 tensor([[2.0346, 0.6980, 0.0920],
         [0.6980, 2.0346, 0.6980],
         [0.0920, 0.6980, 2.0346]], grad_fn=<MulBackward0>))

In [7]:
model.mean_module.state_dict()

OrderedDict([('constant', tensor([-0.1053]))])

In [8]:
model.covar_module.state_dict()

OrderedDict([('raw_outputscale', tensor(1.8945)),
             ('base_kernel.raw_lengthscale', tensor([[-0.8059]])),
             ('base_kernel.lengthscale_prior.concentration', tensor(3.)),
             ('base_kernel.lengthscale_prior.rate', tensor(6.)),
             ('base_kernel.raw_lengthscale_constraint.lower_bound',
              tensor(0.)),
             ('base_kernel.raw_lengthscale_constraint.upper_bound',
              tensor(inf)),
             ('outputscale_prior.concentration', tensor(2.)),
             ('outputscale_prior.rate', tensor(0.1500)),
             ('raw_outputscale_constraint.lower_bound', tensor(0.)),
             ('raw_outputscale_constraint.upper_bound', tensor(inf))])

In [9]:
model.covar_module.raw_outputscale.data = torch.tensor(1.0)

In [10]:
model.covar_module.base_kernel.raw_lengthscale.data = torch.tensor(-1.0)